In [1]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd 
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\91898\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
response = requests.get("https://simple.wikipedia.org/wiki/Climate_change")
soup = BeautifulSoup(response.text, 'html.parser')

links = ["https://simple.wikipedia.org/wiki/Climate_change"]
count=1
for link in soup.findAll('a'):
    links.append(link.get('href'))

In [3]:
links = [x for x in links if x != None]

In [4]:
Internal_Link=[]
External_Link=[]
Link_to_content=[]
for i in range(len(links)):
    print(links[i])
    if(links[i].startswith("/")):
        Internal_Link.append(links[i])
    elif(links[i].startswith("#")):
        Link_to_content.append(links[i])
    elif(links[i].startswith("https")):
        External_Link.append(links[i])

timestamp=response.headers.get('Last-Modified')
data = {'Pagecount':[1],
        'INTcount':[len(Internal_Link)],
        'Extcount':[len(External_Link)],
        'URLfragements':[len(Link_to_content)],
        'timestamp':[timestamp]} 
df = pd.DataFrame(data)

https://simple.wikipedia.org/wiki/Climate_change
#mw-head
#searchInput
/wiki/Global_warming
/wiki/Climate
/wiki/Earth
#cite_note-:0-1
/wiki/Global_warming
#cite_note-:0-1
/wiki/Ice_age
/wiki/Earth%27s_orbit
/wiki/Sun
#cite_note-2
/wiki/Greenhouse_gas
/wiki/Global_warming
#History_of_climate_change_studies
#The_Sun
#Related_pages
#References.
/w/index.php?title=Climate_change&veaction=edit&section=1
/w/index.php?title=Climate_change&action=edit&section=1
/wiki/Joseph_Fourier
/wiki/John_Tyndall
#cite_note-3
/wiki/Svante_Arrhenius
/wiki/Nils_Gustaf_Ekholm
#cite_note-4
#cite_note-5
/w/index.php?title=Climate_change&veaction=edit&section=2
/w/index.php?title=Climate_change&action=edit&section=2
/wiki/Sun
/wiki/Sunspot
/wiki/Stratosphere
/wiki/Global_warming
/w/index.php?title=Climate_change&veaction=edit&section=3
/w/index.php?title=Climate_change&action=edit&section=3
/wiki/Ecology
/w/index.php?title=Climate_change&veaction=edit&section=4
/w/index.php?title=Climate_change&action=edit&secti

In [5]:
def crawl_web(count,url):
    
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    links = []
    for link in soup.findAll('a'):
        links.append(link.get('href'))
    
    links = [x for x in links if x != None]
    
    Internal_Link=[]
    External_Link=[]
    Link_to_content=[]
    for i in range(len(links)):
        if(links[i].startswith("/")):
            Internal_Link.append(links[i])
        elif(links[i].startswith("#")):
            Link_to_content.append(links[i])
        elif(links[i].startswith("https")):
            External_Link.append(links[i])
    timestamp=response.headers.get('Last-Modified')
#     print([[count,len(Internal_Link),len(External_Link),len(Link_to_content),timestamp]])
    to_append=[count,len(Internal_Link),len(External_Link),len(Link_to_content),timestamp]
    df_length = len(df)
    df.loc[df_length] = to_append

In [6]:
for j in range(len(Internal_Link)):
    if(j<200):
        url="https://simple.wikipedia.org"+Internal_Link[j]
        count+=1
        print("Currently processing : "+ url +"\n"+"Page Count" )
        print(count)
        crawl_web(count,url)

Currently processing : https://simple.wikipedia.org/wiki/Global_warming
Page Count
2
Currently processing : https://simple.wikipedia.org/wiki/Climate
Page Count
3
Currently processing : https://simple.wikipedia.org/wiki/Earth
Page Count
4
Currently processing : https://simple.wikipedia.org/wiki/Global_warming
Page Count
5
Currently processing : https://simple.wikipedia.org/wiki/Ice_age
Page Count
6
Currently processing : https://simple.wikipedia.org/wiki/Earth%27s_orbit
Page Count
7
Currently processing : https://simple.wikipedia.org/wiki/Sun
Page Count
8
Currently processing : https://simple.wikipedia.org/wiki/Greenhouse_gas
Page Count
9
Currently processing : https://simple.wikipedia.org/wiki/Global_warming
Page Count
10
Currently processing : https://simple.wikipedia.org/w/index.php?title=Climate_change&veaction=edit&section=1
Page Count
11
Currently processing : https://simple.wikipedia.org/w/index.php?title=Climate_change&action=edit&section=1
Page Count
12
Currently processing : 

In [7]:
df['Pagecount'] = df['Pagecount'].astype(int)
df['INTcount'] = df['INTcount'].astype(int)
df['Extcount'] = df['Extcount'].astype(int)
df['URLfragements'] = df['URLfragements'].astype(int)

In [8]:
df['timestamp'] = pd.to_datetime(df['timestamp'].str[5:], format='%d %b %Y %H:%M:%S GMT')

In [9]:
df['timestamp'].fillna('None', inplace=True)

In [10]:
df

,Pagecount,INTcount,Extcount,URLfragements,timestamp
0,1,68,100,18,2021-11-20 00:42:29
1,2,235,189,93,2021-12-04 04:18:16
2,3,74,153,4,2021-11-20 14:59:08
3,4,808,329,173,2021-12-03 23:01:39
4,5,235,189,93,2021-12-04 04:18:16
...,...,...,...,...,...
64,65,2,2,0,None
65,66,175,200,2,2021-12-01 21:06:39
66,67,100,235,10,2021-11-27 19:43:40
67,68,52,168,2,2021-12-03 21:32:51


In [11]:
df.to_csv("Crawled_data.csv")